# cross match sample

In [ ]:
import hscdata
db = hscdata.connect('pdr1_udeep')

In [ ]:
import numpy
from scipy.spatial import cKDTree as KDTree

def cat2xyz(cat):
    a = numpy.deg2rad(cat.ra)
    d = numpy.deg2rad(cat.dec)
    return numpy.array([
        numpy.cos(d) * numpy.cos(a),
        numpy.cos(d) * numpy.sin(a),
        numpy.sin(d),
    ]).T

def asec2rad(asec):
    return asec / 3600. / 180. * numpy.pi

https://hsc-release.mtk.nao.ac.jp/doc/index.php/data/

In [ ]:
cat = db.forced(where='''
    tract = 9813
    AND patch = 303 -- 3,3 (000 -> 808)
    AND boxSearch(coord, 150, 150.4, 2, 2.4)
    AND detect_is_primary
        AND NOT gcentroid_sdss_flags
        AND NOT rcentroid_sdss_flags
        AND NOT icentroid_sdss_flags
        AND NOT zcentroid_sdss_flags
        AND NOT ycentroid_sdss_flags
        AND NOT gflags_pixel_edge
        AND NOT rflags_pixel_edge
        AND NOT iflags_pixel_edge
        AND NOT zflags_pixel_edge
        AND NOT yflags_pixel_edge
        AND NOT gflags_pixel_interpolated_center
        AND NOT rflags_pixel_interpolated_center
        AND NOT iflags_pixel_interpolated_center
        AND NOT zflags_pixel_interpolated_center
        AND NOT yflags_pixel_interpolated_center
        AND NOT gflags_pixel_saturated_center
        AND NOT rflags_pixel_saturated_center
        AND NOT iflags_pixel_saturated_center
        AND NOT zflags_pixel_saturated_center
        AND NOT yflags_pixel_saturated_center
        AND NOT gflags_pixel_cr_center
        AND NOT rflags_pixel_cr_center
        AND NOT iflags_pixel_cr_center
        AND NOT zflags_pixel_cr_center
        AND NOT yflags_pixel_cr_center
        AND NOT gflags_pixel_bad
        AND NOT rflags_pixel_bad
        AND NOT iflags_pixel_bad
        AND NOT zflags_pixel_bad
        AND NOT yflags_pixel_bad
        AND NOT gcmodel_flux_flags
        AND NOT rcmodel_flux_flags
        AND NOT icmodel_flux_flags
        AND NOT zcmodel_flux_flags
        AND NOT ycmodel_flux_flags
''')

In [ ]:
cat.count()

In [ ]:
import hscMap
w = hscMap.Window(True)

In [ ]:
xyz = cat2xyz(cat)

index = KDTree(xyz)

pairs = index.query_pairs(asec2rad(2))

print len(pairs)

In [ ]:
w.jumpTo({'a': numpy.deg2rad(cat.ra[0]), 'd': numpy.deg2rad(cat.dec[0])})

w.clearMarkers()
# w.addMarker([(cat.ra[i], cat.dec[i]) for i in range(cat.count())]).color = [0, 0.25, 1, 0.5]
w.addMarker([(cat.ra[i], cat.dec[i]) for i, j in pairs])
w.addMarker([(cat.ra[j], cat.dec[j]) for i, j in pairs]).color = [1, 0, 0, 0.5]

In [ ]:
table = [[cat.ra[i], cat.dec[i], i, j] for i, j in pairs]
w.focusTable(['ra', 'dec', 'i', 'j'], table[:100], postageStamp=True)

In [ ]:
def cat2color(cat):
    return numpy.array([
        cat.gmag_kron - cat.imag_kron,
        cat.rmag_kron - cat.imag_kron,
    ]).T

In [ ]:
xyz = cat2xyz(cat)
color = cat2color(cat)

In [ ]:
xyz.shape

In [ ]:
feature = numpy.hstack((xyz / asec2rad(3), color / 0.05))

In [ ]:
index = KDTree(feature)

In [ ]:
pairs = index.query_pairs(1)
print len(pairs)

In [ ]:
pairs = [
    (i, j) for i, j in
    pairs if abs(cat.imag_kron[i] - cat.imag_kron[j])<0.5]

len(pairs)

In [ ]:
len(pairs)

In [ ]:
w.clearMarkers()
# w.addMarker([(cat.ra[i], cat.dec[i]) for i in range(cat.count())]).color = [0, 0.25, 1, 0.01]
w.addMarker([(cat.ra[i], cat.dec[i]) for i, j in pairs])
w.addMarker([(cat.ra[j], cat.dec[j]) for i, j in pairs]).color = [1, 0, 0, 0.5]

In [ ]:
w.filters = ['HSC-Z', 'HSC-R', 'HSC-G']

In [ ]:
table = [[cat.ra[i], cat.dec[i], i, j, cat.imag_kron[i], cat.imag_kron[j]] for i, j in pairs]
w.focusTable(['ra', 'dec', 'i', 'j', 'imag_kron(green)', 'imag_kron(red)'], table, postageStamp=False)